In [69]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [70]:
hn = 3
class Model(nn.Module):
     
    def __init__(self, N):
        super().__init__()
        self.N = N
        self.stepLength = 2        
        self.hidden_initial = torch.Tensor(np.random.normal(0, 1, (1,hn)))
        self.hidden = self.hidden_initial.repeat((self.N, 1))
        
        
        self.Whh = nn.Linear(hn, hn)
        self.Whx = nn.Linear(1, hn, bias = False)
        
        self.layer1 = nn.Linear(1 + hn, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, 1)
  
    
    def forward_DNN(self, x):    
        if self.hidden.shape[0] == x.shape[0]:
            y = torch.cat((self.hidden, x), 1)
        else:
            y = torch.cat((self.hidden[0].repeat(x.shape[0], 1), x), 1)
            
        y = torch.relu(self.layer1(y))
        y = torch.relu(self.layer2(y))
        y = self.layer3(y)
     
        return y
    
    
    def forward_RNN(self, x):
        self.hidden = torch.tanh(self.Whh(self.hidden) + self.Whx(x))
    
    
    def sample(self, N, n):
        total = 0
        x = torch.Tensor(4*np.random.random((N,1)) - 2)
        psi_old = self.forward_DNN(x)
        
        for i in range(n):
            x_new = x + self.stepLength*torch.Tensor(2*np.random.random((N,1)) - 1)
            psi_new = self.forward_DNN(x_new)
            
            idx = (psi_new/psi_old)**2 > torch.Tensor(np.random.random((N,1)))
            
            
            x[idx] = x_new[idx]
            psi_old[idx] = psi_new[idx]
            total += torch.sum(idx)
            
        return x, total
    
    def resetHidden(self):
        self.hidden = self.hidden_initial.repeat((self.N, 1))

In [74]:
N = 10000
n = 20
h = 0.01

torch.manual_seed(42)
np.random.seed(42)
model = Model(N)
print(model.hidden.shape)
optimizer = torch.optim.Adam(model.parameters())

torch.Size([10000, 3])


### Using naive minimization of energy

In [75]:
print(model.sample(N, n)[0][:10])
print(model.sample(N, n)[1])

tensor([[-8.2854],
        [11.2572],
        [-8.4257],
        [ 7.0386],
        [ 3.2015],
        [ 8.0092],
        [-1.5334],
        [ 4.1662],
        [10.4641],
        [-3.4169]])
tensor(168999)


In [79]:
epochs = 2000

for epoch in tqdm(range(epochs)):
    PE_acc = 0
    P_acc = 0
    E_acc = 0
        
    model.resetHidden() 
    x1 = model.sample(N, n)[0].detach()
    psi1 = model.forward_DNN(x1)
    
    model.forward_RNN(x1)
    x2 = model.sample(N, n)[0].detach()
    psi2 = model.forward_DNN(x2)
    
    psi_total = psi1*psi2
    
    model.resetHidden()
    psi1_plus = model.forward_DNN(x1+h)
    model.forward_RNN(x1 + h)
    psi2_plus = model.forward_DNN(x2)
    
    model.resetHidden()
    psi1_minus = model.forward_DNN(x1-h)
    model.forward_RNN(x1 - h)
    psi2_minus = model.forward_DNN(x2)

    lap1 = 1/psi_total*(psi1_plus*psi2_plus - 2*psi_total + psi1_minus*psi2_minus)/h**2
    
    model.resetHidden()
    model.forward_RNN(x1)
    psi2_plus = model.forward_DNN(x2+h)
    psi2_minus = model.forward_DNN(x2-h)
    
    lap2 = 1/psi2*(psi2_plus - 2*psi2 + psi2_minus)/h**2
    

    E_L = (-0.5*(lap1 + lap2) + 0.5*(x1**2 + x2**2) + 1/torch.sqrt((x1 - x2)**2 + 0.1**2)).detach()

    PE = torch.mean(torch.log(psi_total)*E_L)
    P  = torch.mean(torch.log(psi_total))
    E  = torch.mean(E_L)  
    
    loss = 2*(PE - P*E)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    if epoch%100 == 0: 
        print(f"epoch: {epoch}, Energy: {E.item()}")

  0%|          | 2/2000 [00:00<05:18,  6.26it/s]

epoch: 0, Energy: 2.616767168045044


  5%|▌         | 102/2000 [00:15<04:50,  6.54it/s]

epoch: 100, Energy: 2.572415351867676


 10%|█         | 202/2000 [00:30<04:28,  6.70it/s]

...

 90%|█████████ | 1802/2000 [04:31<00:29,  6.73it/s]

epoch: 1800, Energy: 2.5130503177642822


 95%|█████████▌| 1902/2000 [04:46<00:15,  6.42it/s]

epoch: 1900, Energy: 2.5962154865264893


100%|██████████| 2000/2000 [05:01<00:00,  6.62it/s]


In [81]:
def f(x):
    x_lin = torch.linspace(-8, 8, 5000).reshape(5000,-1)
    dx = x_lin[1] - x_lin[0] 
    model.resetHidden()
    psi1 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    x = torch.Tensor([[x]])
    
    model.forward_RNN(x)
    psi2 = model.forward_DNN(x_lin)[:,0].detach().numpy()

    plt.plot(x_lin[:,0], 1/dx*psi1**2/np.sum(psi1**2), "b")
    plt.plot(x_lin[:,0], 1/dx*psi2**2/np.sum(psi2**2), "r")
    plt.plot(x, 0.01, "bo")
    plt.ylim((0, 0.8))

    plt.show()
    
def g(x):
    model.resetHidden()

    x = torch.Tensor([[x]])
    #y = torch.Tensor([[y]])

    plt.plot(model.hidden[0].detach().numpy(), "bo")
    model.forward_RNN(x)
    #model.forward_RNN(y)

    plt.plot(model.hidden[0].detach().numpy(), "ro")
    plt.ylim((-1, 1))

    plt.show()

interact(f, x=(-4.0, 4., 0.05));
interact(g, x=(-4.0, 4., 0.1));

interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0, step=0.05), Output()), _dom_c…

interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0), Output()), _dom_classes=('wi…

In [7]:
N = 10000

pos = []
E = 0
for i in tqdm(range(N)):
    x = model.sample(20)[0][0,0].detach().numpy()
    pos.append(x)

  0%|          | 0/10000 [00:00<?, ?it/s]


TypeError: sample() missing 1 required positional argument: 'n'

In [ ]:
x_lin = torch.linspace(-3, 3, 100).reshape(100,-1)

dx = x_lin[1] - x_lin[0]

psi = model.forward_DNN(x_lin)[:,0].detach().numpy()

bins = np.linspace(-3, 3, 100)
plt.hist(pos, bins=bins)
plt.plot(x_lin[:,0], N*psi**2/np.sum(psi**2), "r")
plt.show()

## Estimating energy

In [ ]:
N = 10000

E = 0
for i in tqdm(range(N)):
    x1 = model.sample(20)[0].detach().requires_grad_()
    psi1 = model.forward_DNN(x1)
        
    dfdx, = torch.autograd.grad(psi1, x1, create_graph=True)
    d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)
        
    kinetic1 = -0.5*d2fdx2/psi1
    potential1 = 0.5*x1**2
        

        
    E += (kinetic1 + potential1).detach()
    
E = E/N

print(E.item())

In [ ]:
x1 = torch.Tensor([[0]]).requires_grad_()
psi1 = model.forward_DNN(x1)


dfdx, = torch.autograd.grad(psi1, x1, create_graph=True)
d2fdx2, = torch.autograd.grad(dfdx, x1, create_graph=True)

print(d2fdx2/psi1)



## Testing auto grad

In [ ]:
x = torch.Tensor([3]).requires_grad_()
y = torch.Tensor([4]).requires_grad_()

f = x**3*y**2

g = f*y
print(f)


dfdx = torch.autograd.grad(g, x, create_graph=True)
print(dfdx)

d2fdx2 = torch.autograd.grad(dfdx, x, create_graph=True)

print(d2fdx2)

#d2fdx2.backward()
#print(x.grad, y.grad)

#print(dfdx)


#ddfdxdx, = torch.autograd.grad(dfdx, x, create_graph=True)
#print(ddfdxdx.grad_fn(x)[1])

In [ ]:
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1, 10)
        self.layer2 = nn.Linear(10, 1)
        
    def forward(self, x):
        y = self.layer1(x).clamp(min=0)
        y = self.layer2(y)
        return y*x

In [ ]:
torch.manual_seed(42)
model = Model()

In [ ]:
#x = torch.Tensor(np.linspace(-0.00001, 0.00001, 100)).reshape(100,-1).requires_grad_()
x = torch.Tensor([1]).requires_grad_()

f = model(x)

ddf = (model(x+0.001) + model(x-0.001) - 2*model(x))/(0.001)**2
print(ddf)

#plt.plot(f.detach().numpy())

dfdx, = torch.autograd.grad(f, x, create_graph = True)
print(dfdx)
ddfdxdx, = torch.autograd.grad(dfdx, x, create_graph = True)
print(ddfdxdx)
